# Import Data - 2 
In this example we see the power of scripting over Excel in that, once we have code for one site, it's fairly easy to repeat the code for many sites.

In [1]:
#Import libraries
import requests, io
import pandas as pd
import numpy as np
import ipyleaflet as lf

In [2]:
#Generate a list of all sites in NC using the NWIS API 
# see https://waterservices.usgs.gov/rest/DV-Service.html
url = 'http://waterservices.usgs.gov/nwis/site'
params = {'stateCd':'NC',        #NC Sites Only
          'siteStatus':'active',    #Get all(active and inactive) sites
          'siteType':'ST',       #Stream sites only
          'parameterCd':'00060', #DISCHARGE(cfs), see https://help.waterdata.usgs.gov/codes-and-parameters/parameters
          'format':'rdb'         #RDB (csv)
         }

In [3]:
#Send the requests and translate the response
response_raw = requests.get(url,params)

In [4]:
#Decode to remove any odd characters (accents, etc.)
response_clean = response_raw.content.decode('utf-8')

In [5]:
#Build a list of line numbers to skip from comments and data types
rowsToSkip= [] #Create an empty list to hold line numbers
lineNumber = 0 #Initialize a line number variable
#Iterate through lines, adding the line number to the list for comment lines
for lineNumber, lineString in enumerate(response_clean.split("\n")):
    if lineString.startswith('#'): 
        rowsToSkip.append(lineNumber)
    else:
        break
#Add another line 2 greater than the last
dataTypeLineNumber = rowsToSkip[-1] + 2
rowsToSkip.append(dataTypeLineNumber)

In [6]:
#Convert the data into a data frame
df = pd.read_csv(io.StringIO(response_clean),
                 skiprows=rowsToSkip,     #Skip metadta and data spec lines
                 delimiter='\t',          #Set to tab delimited
                 dtype={'site_no':'str'}) #Set site_no to a string datatype
#Print the number of returned sites
df.head()

,agency_cd,site_no,station_nm,site_tp_cd,dec_lat_va,dec_long_va,coord_acy_cd,dec_coord_datum_cd,alt_va,alt_acy_va,alt_datum_cd,huc_cd
0,USGS,0204382800,"PASQUOTANK RIVER NEAR SOUTH MILLS, NC",ST,36.421389,-76.342500,S,NAD83,-5.50,0.11,NAVD88,3010205
1,USGS,02053200,"POTECASI CREEK NEAR UNION, NC",ST,36.370833,-77.025556,S,NAD83,3.53,0.01,NGVD29,3010204
2,USGS,02053500,"AHOSKIE CREEK AT AHOSKIE, NC",ST,36.280278,-76.999444,S,NAD83,17.46,0.01,NGVD29,3010203
3,USGS,02068500,"DAN RIVER NEAR FRANCISCO, NC",ST,36.515000,-80.303056,S,NAD83,831.99,0.01,NGVD29,3010103
4,USGS,02069000,"DAN RIVER AT PINE HALL, NC",ST,36.319303,-80.050042,S,NAD83,563.70,0.33,NAVD88,3010103


In [8]:
from ipyleaflet import (
    Map,
    Marker, MarkerCluster,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

In [16]:
center = [36, -80]
zoom = 8

m = Map(default_tiles=TileLayer(opacity=1.0), center=center, zoom=zoom)

for lat,lng,name in df[['dec_lat_va','dec_long_va','station_nm']].values:
    mark = Marker(location=(lat,lng))
    mark.visible
    mark.title = name
    m += mark

m

Map(basemap={'max_zoom': 19, 'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors'}, center=[36, -80], default_tiles=TileLayer(options=['max_zoom', 'opacity', 'min_zoom', 'detect_retina', 'tile_size', 'attribution']), layers=(TileLayer(options=['max_zoom', 'opacity', 'min_zoom', 'detect_retina', 'tile_size', 'attribution']), Marker(location=[36.42138889, -76.3425], options=['keyboard', 'draggable', 'z_index_offset', 'rise_offset', 'rise_on_hover', 'alt', 'opacity', 'clickable', 'title'], title='PASQUOTANK RIVER NEAR SOUTH MILLS, NC'), Marker(location=[36.3708333, -77.0255556], options=['keyboard', 'draggable', 'z_index_offset', 'rise_offset', 'rise_on_hover', 'alt', 'opacity', 'clickable', 'title'], title='POTECASI CREEK NEAR UNION, NC'), Marker(location=[36.28027778, -76.9994444], options=['keyboard', 'draggable', 'z_index_offset', 'rise_offset', 'rise_on_hover', 'alt', 'opacity', 'clickable', 'title'], title='AHOSKIE CREEK AT AHOSKIE, NC'), Marker(location=[36.515, -80.3030556], options=['keyboard', 'draggable', 'z_index_offset', 'rise_offset', 'rise_on_hover', 'alt', 'opacity', 'clickable', 'title'], title='DAN RIVER NEAR FRANCISCO, NC'), Marker(location=[36.31930349, -80.05004240000001], options=['keyboard', 'draggable', 'z_index_offset', 'rise_offset', 'rise_on_hover', 'alt', 'opacity', 'clickable', 'title'], title='DAN RIVER AT PINE HALL, NC'), Marker(location=[36.53388889, -79.99138889], options=['keyboard', 'draggable', 'z_index_offset', 'rise_offset', 'rise_on_hover', 'alt', 'opacity', 'clickable', 'title'], title='MAYO RIVER NEAR PRICE, NC'), Marker(location=[36.4125, -79.82611109999999], options=['keyboard', 'draggable', 'z_index_offset', 'rise_offset', 'rise_on_hover', 'alt', 'opacity', 'clickable', 'title'], title='DAN RIVER NEAR WENTWORTH, NC'), Marker(location=[36.52555556, -79.7655556], options=['keyboard', 'draggable', 'z_index_offset', 'rise_offset', 'rise_on_hover', 'alt', 'opacity', 'clickable', 'title'], title='SMITH RIVER AT EDEN, NC'), Marker(location=[36.39777778, -79.19666670000001], options=['keyboard', 'draggable', 'z_index_offset', 'rise_offset', 'rise_on_hover', 'alt', 'opacity', 'clickable', 'title'], title='HYCO CREEK NEAR LEASBURG, NC'), Marker(location=[36.5225, -78.9975], options=['keyboard', 'draggable', 'z_index_offset', 'rise_offset', 'rise_on_hover', 'alt', 'opacity', 'clickable', 'title'], title='HYCO R BL ABAY D NR MCGEHEES MILL, NC'), Marker(location=[36.540833299999996, -78.8719444], options=['keyboard', 'draggable', 'z_index_offset', 'rise_offset', 'rise_on_hover', 'alt', 'opacity', 'clickable', 'title'], title='MAYO CR NR BETHEL HILL, NC'), Marker(location=[36.46, -77.6336111], options=['keyboard', 'draggable', 'z_index_offset', 'rise_offset', 'rise_on_hover', 'alt', 'opacity', 'clickable', 'title'], title='ROANOKE RIVER AT ROANOKE RAPIDS, NC'), Marker(location=[36.20916667, -77.38388889], options=['keyboard', 'draggable', 'z_index_offset', 'rise_offset', 'rise_on_hover', 'alt', 'opacity', 'clickable', 'title'], title='ROANOKE RIVER NEAR SCOTLAND NECK, NC'), Marker(location=[36.013611100000006, -77.21527778], options=['keyboard', 'draggable', 'z_index_offset', 'rise_offset', 'rise_on_hover', 'alt', 'opacity', 'clickable', 'title'], title='ROANOKE RIVER NEAR OAK CITY, NC'), Marker(location=[35.8597222, -77.04027778], options=['keyboard', 'draggable', 'z_index_offset', 'rise_offset', 'rise_on_hover', 'alt', 'opacity', 'clickable', 'title'], title='ROANOKE RIVER AT WILLIAMSTON, NC'), Marker(location=[36.04777778, -76.9841667], options=['keyboard', 'draggable', 'z_index_offset', 'rise_offset', 'rise_on_hover', 'alt', 'opacity', 'clickable', 'title'], title='CASHIE RIVER AT SR1257 NEAR WINDSOR, NC'), Marker(location=[36.19416667, -78.5830556], options=['keyboard', 'draggable', 'z_index_offset', 'rise_offset', 'rise_on_hover', 'alt', 'opacity', 'clickable', 'title'], title='TAR RIVER NEA